<a href="https://colab.research.google.com/github/ahatasham5/NewspaperSummarizationLLM/blob/main/Newspaper_Sum_with_gemini_1_5_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [17]:

# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [18]:

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [19]:
import keras
import keras_nlp

In [20]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail
License(s): CC0-1.0
newspaper-text-summarization-cnn-dailymail.zip: Skipping, found more recently modified local copy (use --force to force download)


In [23]:
# Unzip the downloaded file
!unzip newspaper-text-summarization-cnn-dailymail.zip

Archive:  newspaper-text-summarization-cnn-dailymail.zip
replace cnn_dailymail/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [26]:
# prompt: i want to load this data set

import pandas as pd

# Specify the path to the extracted directory
data_dir = "/content/cnn_dailymail"

# Load the training data
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))

# Load the validation data
val_df = pd.read_csv(os.path.join(data_dir, "validation.csv"))

# Load the test data
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))


In [29]:
!pip install -q -U google-generativeai

In [33]:
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import pathlib
import textwrap
def to_markdown(text):
  text = text.replace('*', '\\*')
  return Markdown(textwrap.indent(text, '>', predicate=lambda _: True))

In [34]:
from google.colab import userdata

In [35]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [37]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [38]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [39]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
)